# Santa problem from Kaggle 
https://www.kaggle.com/c/traveling-santa-2018-prime-paths/data

You are provided a list of cities and their coordinates in cities.csv. You must create the shortest possible path that visits all the cities. Your submission file is simply the ordered list in which you visit each city. Paths have the following constraints:

Paths must start and end at the North Pole (CityId = 0)
You must visit every city exactly once
The distance between two paths is the 2D Euclidean distance, except...
Every 10th step (stepNumber % 10 == 0) is 10% more lengthy unless coming from a prime CityId.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from prime_numbers import return_list_of_primes, load_primes, save_primes, is_prime
from path_functions import load_best, Generate_Random_Path, is_step_10, dist, length_of_path, dist_primes, length_difference, visualize_path, rearange_cities, MC_step, energy, save_best

In [2]:
# Hyperparameters:
beta = 100 #inverse temperature

# If want to save time
initial_path_length = 447882971.602762 #length of initial full path initiated as range(1,len(cities))
current_best_kaggle = 1514324.54

In [3]:
prime_list = load_primes()
cities = pd.read_csv('cities.csv')
NumberOfCities = len(cities)
cities.head(n=2)

,CityId,X,Y
0,0,316.836739,2202.340707
1,1,4377.405972,336.602082


In [4]:
#path = Generate_Random_Path(cities) # Think about saving these inits to a file for speed
#path = range(1,len(cities))

path, path_length = load_best()

#path_length = length_of_path(path,cities,prime_list)
#path_length = initial_path_length

In [5]:
min_length = path_length
min_path = np.copy(path)

In [6]:
def do_MC(cities,batch_size,path,path_length,prime_list,beta,min_length,min_path):
    for i in range(batch_size):
        #timer function:
        #if (i % (batch_size/10.) == 0):
        #    print(str( (100 * i/batch_size) ) + " percent complete --------- length = " + str(path_length))
        path, path_length = MC_step(cities,path,path_length,prime_list,beta)
        if (path_length < min_length):
            min_length = path_length
            min_path = np.copy(path)
    return path, path_length, min_length, min_path

In [ ]:
#batch_size = len(cities)
minute = 11750
ten_mins = 10*minute #100 000 is about 10 min
hour = 6*ten_mins
batch_size = minute
epochs = 10
SAVE = True
SAVE_EVERY_EPOCH = False

previous_length = path_length

print('started for '+ str(epochs) + ' epochs')

for i in range(epochs):
    previous_epoch_length = path_length
    %time path, path_length, min_length, min_path = do_MC(cities,batch_size,path,path_length,prime_list,beta,min_length,min_path)
    print('epoch:'+str(i+1)+'/'+str(epochs) + '------- path_length = ' + str( path_length.astype(np.int) ) + '--------- Difference: '+ str( (previous_epoch_length - path_length).astype(np.int)) )
    if SAVE_EVERY_EPOCH == True:
        save_best(cities,min_path,min_length) 
    
print('min length = ' + str(min_length.astype(np.int) ) + ',          previously:' + str(previous_length.astype(np.int)) + '------- Total difference:' + str(previous_length-min_length.astype(np.int)))
print('cur length = '+str(path_length.astype(np.int)) )

performance =  current_best_kaggle / min_length
print('performance = ' + str(performance) + ',  (0 is inf bad, 1 is Kaggle best)')

if SAVE:
    save_best(cities,min_path,min_length)  


started for 10 epochs
Wall time: 55.6 s
epoch:1/10------- path_length = 119168338--------- Difference: 28436
Wall time: 53.9 s
epoch:2/10------- path_length = 119147610--------- Difference: 20727
Wall time: 56.9 s
epoch:3/10------- path_length = 119120860--------- Difference: 26750
Wall time: 55 s
epoch:4/10------- path_length = 119097428--------- Difference: 23432
Wall time: 54.9 s
epoch:5/10------- path_length = 119074168--------- Difference: 23259
Wall time: 55.1 s
epoch:6/10------- path_length = 119056275--------- Difference: 17893
Wall time: 56.3 s
epoch:7/10------- path_length = 119038329--------- Difference: 17946
Wall time: 54.3 s
epoch:8/10------- path_length = 119017890--------- Difference: 20438


In [ ]:
#visualize_path(cities,min_path)  #to do: make this useful again by visualizing first 100 steps or sorted

In [17]:
#to do: is_in is very slow. should write a new vector prime_cities = [0, 1, 1, 0, 0,...]

In [ ]:
#to do: try to initialize a path by looking for the shortest distance each time for a next step

In [138]:

find_some_short_path(cities,path)
    

array([ 0, 10,  6,  8,  4,  5, 11,  7,  1,  9,  3,  2])

In [6]:

def find_some_short_path(cities,path):
#no prime stuff in here yet

    temp_path = np.copy(path)
    new_path = np.array([0])

    while (len(temp_path) > 0 ):
        print(len(temp_path))
        distances = [dist(cities,new_path[-1],temps) for temps in temp_path]
        new_path = np.append( new_path, temp_path[np.argmin(distances)] )
        temp_path = np.delete( temp_path, np.argmin(distances)  )
    return new_path

In [ ]:
find_some_short_path(cities,path)

197768


In [32]:
%time distances = [dist2(0,temps) for temps in path]

Wall time: 1min 37s
